Construct Unitary Matrix

In [2]:
import pennylane as qml
import numpy as np
import jax
import cirq
from qiskit.quantum_info import *
from qiskit.synthesis import OneQubitEulerDecomposer
from scipy.linalg import logm
from scipy.linalg import expm
from scipy.optimize import minimize


In [3]:
wires = [0, 1]

# Activating the Pauli words ["IY", "IZ", "XX", "XY", "YY", "YZ", "ZY", "ZZ"]

#init_params = 0.3 * np.array([0, 1, 2, 0, -1, 1, 0, 0, 0, 1, 1, 1, 0, 0, -1])
init_params = np.ones(15)
# len(init_params) == 4 ** len(wires) - 1 # theta contains one parameter per Pauli word

#init_params = np.random.uniform(0, 2*np.pi, size=15)

su = qml.SpecialUnitary(init_params, wires=wires)
matrix = su.matrix()



np.linalg.det(matrix)
# A_original = logm(matrix)
#assert np.allclose(matrix, matrix.conjugate().T), "SU(N) is not Hermitian"

np.allclose(np.conjugate(matrix.T), -matrix, atol=1e-6)

matrix


array([[ 0.32179375+0.04488872j,  0.42619418-0.24261069j,
         0.42619418-0.24261069j,  0.00242523-0.64298138j],
       [ 0.24261069+0.42619418j, -0.34701112-0.13869478j,
         0.64298138+0.00242523j, -0.21921243+0.39794546j],
       [ 0.24261069+0.42619418j,  0.64298138+0.00242523j,
        -0.34701112-0.13869478j, -0.21921243+0.39794546j],
       [-0.00242523+0.64298138j, -0.39794546-0.21921243j,
        -0.39794546-0.21921243j,  0.27014678-0.31742781j]])

# Extraction of the coefficients

## Matrix Logarithm:
Given a unitary matrix \( U \in SU(2^n) \), we can write \( U \) in terms of its matrix logarithm as:

$$
U = e^{A}
$$

where \( A \) is a Hermitian matrix. This matrix \( A \) can be expanded in the Pauli basis:

$$
A = \sum_k i c_k P_k = ln(U)

$$

where $$P_k$$ are the Pauli matrices or tensor products of Pauli matrices, and \( c_k \) are real coefficients.

## Finding the Coefficients:
To find the coefficients \( i c_k \), we use the orthogonality property of the Pauli matrices. For a Pauli matrix \( P_k \):

$$
c_k = \frac{1}{2^n} \text{Tr}[A P_k]
$$

This formula arises because the trace operation is linear, and Pauli matrices are orthogonal under the trace operation.


In [4]:
# Define the Pauli matrices
I = np.eye(2)
sigma_x = np.array([[0, 1], [1, 0]])
sigma_y = np.array([[0, -1j], [1j, 0]])
sigma_z = np.array([[1, 0], [0, -1]])
pauli_basis = {
    'IX': np.kron(I, sigma_x), 'IY': np.kron(I, sigma_y), 'IZ': np.kron(I, sigma_z),
    'XI': np.kron(sigma_x, I), 'XX': np.kron(sigma_x, sigma_x), 'XY': np.kron(sigma_x, sigma_y), 'XZ': np.kron(sigma_x, sigma_z),
    'YI': np.kron(sigma_y, I), 'YX': np.kron(sigma_y, sigma_x), 'YY': np.kron(sigma_y, sigma_y), 'YZ': np.kron(sigma_y, sigma_z),
    'ZI': np.kron(sigma_z, I), 'ZX': np.kron(sigma_z, sigma_x), 'ZY': np.kron(sigma_z, sigma_y), 'ZZ': np.kron(sigma_z, sigma_z)
}

In [5]:
from scipy.linalg import logm

def extract_pauli_coeff(U,threshold=1e-8):
    # Define the Pauli matrices
    I = np.eye(2)
    sigma_x = np.array([[0, 1], [1, 0]])
    sigma_y = np.array([[0, -1j], [1j, 0]])
    sigma_z = np.array([[1, 0], [0, -1]])

    # Compute the Hermitian matrix H from the matrix logarithm of U
    H = logm(U)

    # Define the extended Pauli basis for two qubits
    pauli_basis = {
        'IX': np.kron(I, sigma_x), 'IY': np.kron(I, sigma_y), 'IZ': np.kron(I, sigma_z),
        'XI': np.kron(sigma_x, I), 'XX': np.kron(sigma_x, sigma_x), 'XY': np.kron(sigma_x, sigma_y), 'XZ': np.kron(sigma_x, sigma_z),
        'YI': np.kron(sigma_y, I), 'YX': np.kron(sigma_y, sigma_x), 'YY': np.kron(sigma_y, sigma_y), 'YZ': np.kron(sigma_y, sigma_z),
        'ZI': np.kron(sigma_z, I), 'ZX': np.kron(sigma_z, sigma_x), 'ZY': np.kron(sigma_z, sigma_y), 'ZZ': np.kron(sigma_z, sigma_z)
    }

    # Calculate the coefficients
    # coefficients = {name: np.trace(H @ Pk)/ (4j) for name, Pk in pauli_basis.items()}
    coefficients = {}
    for key, Pk in pauli_basis.items():
        coeff = np.trace(H @ Pk) / 4
        # Set coefficients below the threshold to zero
        if abs(coeff.real) < threshold:
            coeff = complex(0, coeff.imag)
        if abs(coeff.imag) < threshold:
            coeff = complex(coeff.real, 0)
        coefficients[key] = coeff
    return coefficients

extract_pauli_coeff(matrix)

{'IX': 0.09310031788289097j,
 'IY': 0.09310031788289086j,
 'IZ': 0.09310031788289119j,
 'XI': 0.09310031788289078j,
 'XX': 0.47640122440170096j,
 'XY': 0.47640122440170124j,
 'XZ': 0.47640122440170113j,
 'YI': 0.09310031788289094j,
 'YX': 0.4764012244017011j,
 'YY': 0.47640122440170113j,
 'YZ': 0.4764012244017012j,
 'ZI': 0.09310031788289186j,
 'ZX': 0.47640122440170096j,
 'ZY': 0.4764012244017012j,
 'ZZ': 0.47640122440170096j}

# Consistency Check

To check if these complex coefficients generate the same expectation values as other parameterization methods, you can:

## 1. Reconstruct A
Use your coefficients to reconstruct \( A \).

$$ 
A = \sum_{k} i c_{k} P_{k} 
$$

## 2. Calculate Expectation Values
For a given state \( \rho \), calculate the expectation value using both the original and reconstructed Hamiltonian.

$$ 
\langle A \rangle_{\text{original}} = \operatorname{Tr}(\rho A_{\text{original}}) 
$$

$$ 
\langle A \rangle_{\text{reconstructed}} = \operatorname{Tr}\left(\rho \sum_{k} i c_{k} P_{k}\right) 
$$

## 3. Compare Results
Ensure that the expectation values are the same for the original and reconstructed Hamiltonian.


In [6]:
coefficients = extract_pauli_coeff(matrix)

def reconstruct_A(coefficients):
    A_reconstructed = sum(coefficients[key] * pauli_basis[key] for key in coefficients)
    
    return A_reconstructed

# Define a test density matrix (example)
#rho = np.kron(np.array([[1, 0], [0, 0]]), np.array([[0, 0], [0, 1]]))
rho = np.array([
    [1, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0]
], dtype=complex)

A_reconstructed = reconstruct_A(coefficients)

# Compute expectation values
expectation_original = np.trace(np.dot(rho,logm(matrix)))#np.trace(rho @ logm(matrix))

expectation_reconstructed = np.trace(np.dot(rho,A_reconstructed))#np.trace(rho @ A_reconstructed)

print(f"Original expectation value: {expectation_original}")
print(f"Reconstructed expectation value: {expectation_reconstructed}")


#assert np.allclose(A_reconstructed, A_reconstructed.conjugate().T), "A is not Hermitian"


Original expectation value: (-1.1102230246251565e-16-0.9081944666274131j)
Reconstructed expectation value: 0.662601860167484j


In [7]:
# Check if the expectation values are close within a numerical threshold
threshold = 1e-8
if np.abs(expectation_original - expectation_reconstructed) < threshold:
    print("Expectation values are consistent within the threshold.")
else:
    print("Expectation values are not consistent within the threshold.")

Expectation values are not consistent within the threshold.


#STEP 2

### Step 2: Initialize Parameters for Pauli Rotations(2 Qubit Decomp) and Obtain Equivalent Parameters in the \(SU(N)\) Picture

To initialize the parameters for Pauli rotations, we use random values drawn from a uniform distribution over the interval $$[0, 2\pi]$$. This step is crucial as it sets the initial state for our system in a way that covers the entire parameter space.


In [8]:
# Define the device
dev = qml.device('default.qubit', wires=2)

# Define the circuit
def two_qubit_decomp_1(params, wires):
    """Implement an arbitrary SU(4) gate on two qubits
    using the decomposition from Theorem 5 in
    https://arxiv.org/pdf/quant-ph/0308006.pdf"""
    
    i, j = wires    
    qml.RZ(params[0], wires=i)
    qml.RY(params[1], wires=i)
    qml.RZ(params[2], wires=i)
    qml.RZ(params[3], wires=j)
    qml.RY(params[4], wires=j)
    qml.RZ(params[5], wires=j)
    qml.CNOT(wires=[j, i])
    qml.RZ(params[6], wires=i)
    qml.RY(params[7], wires=j)
    qml.CNOT(wires=[i, j])
    qml.RY(params[8], wires=j)
    qml.CNOT(wires=[j, i])
    qml.RZ(params[9], wires=j)
    qml.RY(params[10], wires=j)
    qml.RZ(params[11], wires=j)
    qml.RZ(params[12], wires=i)
    qml.RY(params[13], wires=i)
    qml.RZ(params[14], wires=i)

# Wrap the circuit in a QNode
@qml.qnode(dev)
def circuit(params):
    two_qubit_decomp_1(params, wires=[0, 1])
    return qml.state()

# Example parameters
init_pauli_params = np.ones(15)##np.random.uniform(0, 2*np.pi, size=15)

# Extract the unitary matrix
U_rot = qml.matrix(circuit)(init_pauli_params)

# Print the unitary matrix
print("Unitary matrix U:\n", U_rot)


type(U_rot)

np.allclose(U_rot.conj().T @ U_rot, np.eye(4))

#np.isclose(np.linalg.det(U_rot), -1,atol = 1e-8)
np.linalg.det(U_rot)

Unitary matrix U:
 [[-0.06754741+0.65273498j -0.01473623+0.42577198j -0.01473623+0.42577198j
  -0.15815391+0.42586836j]
 [-0.0353509 +0.2805277j  -0.24083787-0.20789815j  0.63674469+0.27152739j
  -0.18282119-0.55338548j]
 [-0.0353509 +0.2805277j   0.63674469+0.27152739j -0.24083787-0.20789815j
  -0.18282119-0.55338548j]
 [ 0.63982965-0.01007205j -0.4385717 +0.213917j   -0.4385717 +0.213917j
   0.00857712-0.3379819j ]]


(-1.0000000000000022+5.551115123125795e-17j)

# Extract Coefficient from the 2_qubit_decomp Unitary

In [9]:
coeff_Urot = extract_pauli_coeff(U_rot)

A_reconstructed = reconstruct_A(coeff_Urot)

U_SUN_rec = expm(A_reconstructed)

U_SUN_rec

np.linalg.det(U_SUN_rec)


(0.9999999999999999+3.8857805861880474e-16j)

In [10]:
rho = np.array([
    [1, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0]
], dtype=complex)

# Compute expectation values
expectation_original = np.trace(np.dot(rho,U_rot))#np.trace(rho @ logm(matrix))

expectation_reconstructed = np.trace(np.dot(rho,U_SUN_rec))#np.trace(rho @ A_reconstructed)

print(f"Original expectation value: {expectation_original}")
print(f"Reconstructed expectation value: {expectation_reconstructed}")

print(np.absolute(expectation_original),np.absolute(expectation_reconstructed))

Original expectation value: (-0.06754741058426489+0.652734982957205j)
Reconstructed expectation value: (-0.5093165648424507+0.4137901006909998j)
0.6562207027005335 0.6562207027005337


# Comparing the expectation values using a two qubit Hamiltonian



In [11]:
# Define 2 qubit hamiltonian

# Define the Pauli matrices
I = np.eye(2)
sigma_x = np.array([[0, 1], [1, 0]])
sigma_y = np.array([[0, -1j], [1j, 0]])
sigma_z = np.array([[1, 0], [0, -1]])

# Define the coefficients for the Hamiltonian terms
J = 1.0  # Interaction strength
B = 0.5  # External magnetic field

# Construct the Hamiltonian
H = (J * np.kron(sigma_x, sigma_x) +
     J * np.kron(sigma_y, sigma_y) +
     J * np.kron(sigma_z, sigma_z) +
     B * np.kron(sigma_z, I) +
     B * np.kron(I, sigma_z))

# Print the Hamiltonian
print("Hamiltonian:")
print(H)

Hamiltonian:
[[ 2.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j  2.+0.j  0.+0.j]
 [ 0.+0.j  2.+0.j -1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]]


## Transforming an Initial State \(|00\rangle\) Under a Unitary Transformation

### Initial State \(|00\rangle\)

The initial state \(|00\rangle\) for two qubits can be represented as a column vector:
$$
|00\rangle = \begin{pmatrix}
1 \\
0 \\
0 \\
0 
\end{pmatrix}
$$

### Unitary Transformation

Given a unitary matrix \(U\), the transformed state \(|\psi\rangle\) is obtained by multiplying \(U\) with the initial state \(|00\rangle\):
$$
|\psi\rangle = U |00\rangle
$$

### Density Matrix \(\rho\)

The density matrix \(\rho\) of the transformed state \(|\psi\rangle\) is given by:
$$
\rho = |\psi\rangle \langle \psi|
$$

This can be computed as:
$$
\rho = U |00\rangle \langle 00| U^\dagger
$$

Where \(U^\dagger\) is the conjugate transpose of \(U\).

### Summary

1. **Initial State \(|00\rangle\)**: Represented as a column vector \(\begin{pmatrix} 1 & 0 & 0 & 0 \end{pmatrix}^T\).

2. **Unitary Transformation**: Multiply the initial state by the unitary matrix \(U\) to obtain the transformed state \(|\psi\rangle\):
   $$
   |\psi\rangle = U |00\rangle
   $$

3. **Density Matrix \(\rho\)**: Compute the density matrix \(\rho\) as the outer product of \(|\psi\rangle\) with its conjugate transpose:
   $$
   \rho = |\psi\rangle \langle \psi|
   $$

By applying these steps, we can understand how an initial state \(|00\rangle\) transforms under a unitary operation and obtain the corresponding density matrix.


In [25]:
import numpy as np

def transform_and_get_density_matrix(U):
    # Define the initial state |00>
    initial_state = np.array([1, 0, 0, 0], dtype=complex).reshape(-1, 1)

    # Apply the unitary transformation
    transformed_state = np.dot(U, initial_state)

    # Compute the density matrix
    rho = np.dot(transformed_state, transformed_state.conj().T)

    return rho

# Example usage
# Define a unitary matrix U (example: 4x4 identity matrix)
U = np.eye(4, dtype=complex)

# Transform the initial state |00> under U and get the density matrix
rho = transform_and_get_density_matrix(U)

# Print the resulting density matrix
print("Density Matrix ρ:")
print(rho)


rho_t = transform_and_get_density_matrix(U_SUN_rec)

np.trace(rho_t)

Density Matrix ρ:
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]


(1+0j)

In [26]:
# Compute expectation values
expectation_original = np.trace(np.dot(transform_and_get_density_matrix(U_rot),H))#np.trace(rho @ logm(matrix))

expectation_reconstructed = np.trace(np.dot(transform_and_get_density_matrix(U_SUN_rec),H))#np.trace(rho @ A_reconstructed)

print(f"Original expectation value: {expectation_original}")
print(f"Reconstructed expectation value: {expectation_reconstructed}")

print(np.absolute(expectation_original),np.absolute(expectation_reconstructed))

Original expectation value: (1.0211421786396893+0j)
Reconstructed expectation value: (1.0211421786396895+0j)
1.0211421786396893 1.0211421786396895


In [20]:
def is_skew_hermitian(matrix, tol=1e-10):
    """
    Check if a matrix is skew-Hermitian.
    
    Parameters:
    matrix (numpy.ndarray): The matrix to be checked.
    tol (float): Tolerance for numerical precision.
    
    Returns:
    bool: True if the matrix is skew-Hermitian, False otherwise.
    """
    # Calculate the conjugate transpose of the matrix
    matrix_dagger = np.conjugate(matrix.T)
    
    # Check if the matrix satisfies the skew-Hermitian condition: A_dagger = -A
    return np.allclose(matrix_dagger, -matrix, atol=1e-6)


is_skew_hermitian(H)

False

In [13]:

pauli_basis_1 = [
    np.kron(I, sigma_x), np.kron(I, sigma_y), np.kron(I, sigma_z),
    np.kron(sigma_x, I), np.kron(sigma_x, sigma_x), np.kron(sigma_x, sigma_y), np.kron(sigma_x, sigma_z),
    np.kron(sigma_y, I), np.kron(sigma_y, sigma_x), np.kron(sigma_y, sigma_y), np.kron(sigma_y, sigma_z),
    np.kron(sigma_z, I), np.kron(sigma_z, sigma_x), np.kron(sigma_z, sigma_y), np.kron(sigma_z, sigma_z)
]



#Construct A

In [14]:
def random_pauli_rotations():
    # Initialize parameters for the Pauli rotations
    return np.random.uniform(0, 2 * np.pi, size=15)  # For SU(4), we need 15 parameters



def construct_unitary(params):
    # Construct the unitary matrix from the parameters
    A = sum(1j*theta_k * P_k for theta_k, P_k in zip(params, pauli_basis_1))
    return expm(A)


pauli_init_params = random_pauli_rotations()

U = construct_unitary(pauli_init_params)



In [15]:
def is_special_unitary(matrix, tol=1e-8):
    """
    Check if a matrix belongs to the special unitary group SU(4).

    Parameters:
    matrix (np.ndarray): The matrix to check.
    tol (float): Tolerance for numerical precision.

    """
    # Check if the matrix is unitary: U^dagger U = I
    if not np.allclose(matrix.conj().T @ matrix, np.eye(4), atol=tol):
        return False
    
    # Check if the determinant is 1
    if not np.isclose(np.linalg.det(matrix), 1, atol=tol):
        return False
    
    return True
is_special_unitary(U)

True

In [16]:
U

array([[ 6.66409089e-01+0.3569677j ,  9.01270973e-02-0.02374948j,
         1.46032413e-01+0.55706264j,  1.35188810e-01-0.26432137j],
       [-1.47064307e-04+0.01066682j,  6.14222879e-01+0.71675118j,
         2.31276343e-01-0.05414468j,  8.10985097e-02+0.21421212j],
       [-6.97646939e-02+0.6137441j , -1.13223155e-01-0.17921089j,
         6.20213657e-01-0.30125933j, -2.13813298e-01+0.22885967j],
       [-2.10677873e-01+0.04924509j, -1.64554267e-01+0.16821692j,
         2.58090965e-01-0.26439032j,  5.74584902e-01-0.65662296j]])

In [17]:
def cost_function(params, target_unitary):
    U = construct_unitary(params)
    # Fidelity cost (1 - Fidelity)
    fidelity = np.abs(np.trace(np.dot(target_unitary.conj().T, U))) / 4
    return 1 - fidelity

# Example target unitary (identity matrix here)
target_unitary = np.eye(4)

# Initialize parameters
initial_params = random_pauli_rotations()

# Optimize the parameters
result = minimize(cost_function, initial_params, args=(target_unitary,), method='BFGS')

# Optimized parameters
optimized_params = result.x

# Construct the optimized unitary
optimized_unitary = construct_unitary(optimized_params)

# Check fidelity
fidelity = np.abs(np.trace(np.dot(target_unitary.conj().T, optimized_unitary))) / 4
print(f"Fidelity: {fidelity}")

Fidelity: 0.9999999999999352
